# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install opencv-python-headless ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 29.1 MB/s eta 0:00:00


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [91]:
from ultralytics import YOLO
model = YOLO("/content/YOLO_Model.pt")

# model = YOLO("yolov8n.pt")


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [92]:
import cv2

cap = cv2.VideoCapture("/content/YOLO_Video.mp4")

if not cap.isOpened():
  print(" no file")



## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [71]:
frame_width = int(cap.get(3))
frame_hight = int(cap.get(4))
fps = int(cap.get(5))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [72]:
out = cv2.VideoWriter("exam_output.avi",cv2.VideoWriter_fourcc(*"MJPG"),fps,(frame_width,frame_hight))


## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [93]:
top_cars = 0
bottom_cars = 0
right_cars = 0
left_cars = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame)
    frame_height, frame_width = frame.shape[:2]

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            if y2 > frame_height * 0.7 and y1 > frame_height and x1>frame_width*0.4 and x2>frame_width*0.9:
                bottom_cars += 1
            elif y1 < frame_height * 0.3 and y2 > frame_height * 0.1 and x1 > frame_width * 0.2 and x2 > frame_width *0.6:
                top_cars += 1
            elif x2 > frame_width * 0.8 and x1 > frame_width and y1 > frame_height * 0.6 and y2 > frame_height * 0.3:
                right_cars += 1
            elif x1 < frame_width * 0.2 and x2 < frame_width * 0.1 and y1 > frame_height * 0.5 and y2 > frame_height * 0.2:
                left_cars += 1

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 52.9ms
Speed: 3.5ms preprocess, 52.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 41.5ms
Speed: 3.3ms preprocess, 41.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 40.4ms
Speed: 3.0ms preprocess, 40.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 37.4ms
Speed: 3.3ms preprocess, 37.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 31.9ms
Speed: 3.0ms preprocess, 31.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 31.2ms
Speed: 3.3ms preprocess, 31.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 32.4ms
Speed: 3.5ms preprocess, 32.4ms inferenc

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [94]:
print (top_cars)
print (bottom_cars)
print (right_cars)
print (left_cars)



5515
0
0
1877


## Save and Submit Your Work

Submit both the notebook and the output video

In [ ]:
# OUTPUT FILE ON DRIVE 

#   https://drive.google.com/file/d/1aY_Cu8pl3hUXNSsy8DKmNuaAyMvooC1M/view?usp=drive_link

In [ ]:
from google.colab import files
files.download("exam_output.avi")